In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
from pycaret.regression import *
import category_encoders as  ce
from sklearn.preprocessing import normalize


In [ ]:
df_train_X = pd.read_csv("../input/epidemy/dengue_features_train.csv")
df_train_Y = pd.read_csv("../input/epidemy/dengue_labels_train.csv")
df_test = pd.read_csv("../input/epidemy/dengue_features_test.csv")
df_test_copy = df_test.copy()

In [ ]:
df_train_X

In [ ]:
df_train_X.isnull().sum()

In [ ]:
for column in df_train_X.columns:
    if df_train_X[column].isnull().sum()!=0:
        print(df_train_X[column].describe(),"\n\n")

In [ ]:
for column in df_train_X.columns:
    if df_train_X[column].isnull().sum()!=0:
        df_train_X[column].fillna(np.mean(df_train_X[column]),inplace=True)
    else:
        print(column)

In [ ]:
df_train_X.isnull().sum()

In [ ]:
df_train = df_train_X.copy()


In [ ]:
df_train_X

In [ ]:
def ordinal_encoding(df,col,mapping):
    ordinal_encoder = ce.OrdinalEncoder(cols = [col],return_df = True,mapping = [{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final


In [ ]:
df_train = ordinal_encoding(df_train,"city",{"sj":0,"iq":1})
df_test = ordinal_encoding(df_test,"city",{"sj":0,"iq":1})

In [ ]:
df_train[["start_year","start_month","start_date"]] = df_train["week_start_date"].str.split("-",n=3,expand =True)
df_train.drop(["week_start_date"],axis=1,inplace = True)


In [ ]:
df_train

In [ ]:
for column in df_test.columns:
    if df_test[column].isnull().sum()!=0:
        df_test[column].fillna(np.mean(df_test[column]),inplace=True)
    else:
        print(column)

In [ ]:
df_test[["start_year","start_month","start_date"]] = df_test["week_start_date"].str.split("-",n=3,expand =True)
df_test.drop(["week_start_date"],axis=1,inplace = True)

In [ ]:
set(df_train.columns)-set(df_test.columns)

In [ ]:

df_train_cleaned = normalize(df_train)
df_test_cleaned = normalize(df_test)


df_train = pd.DataFrame(df_train_cleaned,columns=df_train.columns)
df_train["total_cases"] = df_train_Y["total_cases"]
df_test = pd.DataFrame(df_test_cleaned,columns=df_test.columns)

In [ ]:

experiment = setup(data = df_train,target = "total_cases")

In [ ]:
compare_models()

In [ ]:
lightgbm = create_model("lightgbm")

In [ ]:
y_pred = predict_model(lightgbm,data=df_test)

In [ ]:
def roundVal(val):
    return abs(round(val))
y_pred["total_cases"] = y_pred["Label"].apply(roundVal)

In [ ]:
y_pred["total_cases"]

In [ ]:
submission = pd.DataFrame()
submission["city"] = df_test_copy["city"]
submission["year"] = df_test_copy["year"]
submission["weekofyear"] = df_test_copy["weekofyear"]
submission["total_cases"] = y_pred["total_cases"]
submission.to_csv("submission.csv",index= False)